# API

In [ ]:
#| default_exp serve

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
import subprocess
from time import time

## `serve_dev()`

The `serve_dev()` function is a wrapper around the FastHTML `serve()` function, adding utilities to enhance the development experience.

In [ ]:
#| export
def serve_dev(
    app='app',                 # FastHTML (Starlette) class instance
    host='0.0.0.0',            # Uvicorn host
    port=None,                 # Uvicorn port
    reload=True,               # Uvicorn reload status
    reload_includes=None,      # Files to watch to reload Uvicorn
    reload_excludes=None,      # Files to exclude from watching to reload Uvicorn
    sqlite_port=8035,          # sqlite-web port
    db=False,                  # Enable SQLIte browser
    db_path='data/app.db',     # SQLite database path
    jupyter=False,             # Enable Jupyter Lab
    jupyter_port=8036,         # Jupyter Lab port
    tw=False,                  # Enable TailwindCSS
    tw_src='./app.css',        # TailwindCSS source file
    tw_dist='./public/app.css' # TailwindCSS output file
):
    "Utility function to start FastHTML, TailwindCSS, Jupyter Lab, and SQLite in development mode"
    import inspect
    frame = inspect.currentframe().f_back
    module = inspect.getmodule(frame)
    
    # Check if this is the main module
    if module.__name__ != '__main__':
        return

    appname = module.__name__

    if tw:
        print("Watching for TailwindCSS class changes...")
        tailwind_process = subprocess.Popen(
            ['tailwindcss', '-i', tw_src, '-o', tw_dist, '--watch'],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )

    if db:
        #print("Starting SQLite...")
        sqlite_process = subprocess.Popen(
            ['sqlite_web', db_path, '--port', str(sqlite_port), '--no-browser'],
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL
        )
        print(f'SQLite link: http://localhost:{sqlite_port}')

    if jupyter:
        #print("Starting Jupyter...")
        jupyter_process = subprocess.Popen(
            ['jupyter', 'lab', '--port', str(jupyter_port), '--no-browser', '--NotebookApp.token=', '--NotebookApp.password='],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
            text=True
        )

        # Extract and print the Jupyter Lab URL
        for line in jupyter_process.stderr:
            if 'http://' in line:
                match = re.search(r'(http://localhost:\d+/lab)', line)
                if match:
                    print(f'Jupyter Lab link: {match.group(1)}')
                    break

    try:
        #print("Starting FastHTML...")
        serve(appname=appname, app=app, host=host, port=port, reload=reload, reload_includes=reload_includes, reload_excludes=reload_excludes)
    finally:
        if db:
            sqlite_process.terminate()
        if jupyter:
            jupyter_process.terminate()
        if tw:
            tailwind_process.terminate()


## `cache_buster()`

It can be annoying sometimes when FastHTML live reload is enabled and you're quickly iterating on CSS changes and the styles sometimes get cached between page loads. When this happens you have to manually clear the browser cache and reload the page.

The `cache_buster()` function adds a unique timestamp string to the CSS file added to the site header which is different every page load, so this should eliminate browser caching of CSS leading to a smoother development workflow.

In [ ]:
#| export
def cache_buster():
	"Helps to avoid browser caching of dynamically generated CSS during development"
	return f"?v={int(time())}"

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()